In [1]:
from polls import *
from UKGE import *
import numpy as np

In [2]:
df = pd.read_csv("outputs/resultsclusteredconstituencies.csv")

In [3]:
#Predicting 2019
poll_avg19 = get_weighted_poll_avg(url_19, col_dict=col_dict19)
poll_avg17 = get_weighted_poll_avg(url_17, col_dict=col_dict17)
poll_avg15 = get_weighted_poll_avg(url_15, col_dict=col_dict15)
poll_avg10 = get_weighted_poll_avg(url_10, col_dict=col_dict10)
poll_avg05 = get_weighted_poll_avg(url_05, col_dict=col_dict05)
poll_avg19 = standardise_df(poll_avg19, col_dict19)
poll_avg17 = standardise_df(poll_avg17, col_dict17)
poll_avg15 = standardise_df(poll_avg15, col_dict15)
poll_avg10 = standardise_df(poll_avg10, col_dict10)
poll_avg05 = standardise_df(poll_avg05, col_dict05)



In [4]:

gb19 = get_GB_res(df)
pc_res_19 = convert_to_pc(gb19)
gb17 = get_GB_res(df,yr=2017)
pc_res_17 = convert_to_pc(gb17)
gb15 = get_GB_res(df,yr=2015)
pc_res_15 = convert_to_pc(gb15)
gb10 = get_GB_res(df,yr=2010)
pc_res_10 = convert_to_pc(gb10)
gb05 = get_GB_res(df,yr=2005)
pc_res_05 = convert_to_pc(gb05)

In [5]:
poll_avgs = [poll_avg05,poll_avg10,poll_avg15,poll_avg17,poll_avg19]
pc_results = [pc_res_05,pc_res_10,pc_res_15,pc_res_17,pc_res_19]

In [6]:
bigerrs = []
for p,r in zip(poll_avgs,pc_results):
    conerr = float(p["con"] - r["con"])
    laberr = float(p["lab"] - r["lab"])

    bigerrs.append(conerr)
    bigerrs.append(laberr)






/tmp/ipykernel_35709/1519193469.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  conerr = float(p["con"] - r["con"])
/tmp/ipykernel_35709/1519193469.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  laberr = float(p["lab"] - r["lab"])


In [7]:
big_std = np.std(bigerrs)

In [8]:
pc_res_05

,con,lab,lib,ref,nat,oth
0,0.332274,0.36199,0.226456,0.0,0.022213,0.057067


In [9]:
pd.DataFrame(poll_avg05).transpose().columns

Index(['Con', 'Lab', 'Lib Dem', 'Others', 'con', 'lab', 'lib', 'oth'], dtype='object')

In [10]:
serrs = []
for p,r in zip(poll_avgs,pc_results):
    liberr = float(p["lib"] - r["lib"])
    serrs.append(liberr)
    if "ref" in pd.DataFrame(p).transpose().columns:
        referr = float(p["ref"] - r["ref"])
        serrs.append(referr)
        
    if "nat" in pd.DataFrame(p).transpose().columns:
        naterr = float(p["nat"] - r["nat"])
        serrs.append(naterr)




/tmp/ipykernel_35709/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_35709/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_35709/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_35709/2919726663.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  referr = float(p["ref"] - r["ref"])
/tmp/ipykernel_35709/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a

In [11]:
small_std = np.std(serrs)

In [12]:
constituencies = list(df["New constituency name"])

In [13]:
next_poll_avg = get_weighted_poll_avg(next_url, col_dict=next_col_dict)
nat_polls = standardise_df(next_poll_avg, next_col_dict)
chg_df = nat_polls - pc_res_19
chg_df = chg_df.dropna(axis=1).copy()

In [14]:
chg_df

,con,lab,lib,nat,oth,ref
0,-0.225489,0.109106,-0.03195,-0.015484,0.016707,0.086444


In [15]:
natchgs = []
sims_num = 1000
future_uncertainty = 3
for p in std_parties:
    if nat_polls[p] > 0.2:
        natchgs.append(np.random.normal(chg_df[p],big_std*future_uncertainty,sims_num))
    else:
        natchgs.append(np.random.normal(chg_df[p],small_std*future_uncertainty,sims_num))
clusters = list(set(list(df["Cluster"])))
for p in std_parties:
    for c in clusters:
        if nat_polls[p] > 0.2:
            natchgs.append(np.random.normal(0,big_std,sims_num))
        else:
            natchgs.append(np.random.normal(0,small_std,sims_num))

In [16]:
chg_dict = {}


In [17]:
i=0
for p in std_parties:
    chg_dict[p+"n"] = natchgs[i]
    i+=1
for c in clusters:
    for p in std_parties:
        chg_dict[p+str(c)] = natchgs[i]
        i+=1

In [18]:
rand_df = pd.DataFrame(chg_dict)

In [19]:
# Create a dictionary to store the results for each constituency and party
results = {}

# Iterate over constituencies
for con in constituencies:
    # Filter the dataframe for the current constituency
    con_df = df[df["New constituency name"] == con].copy()
    
    # Get the cluster, imputation uncertainty, and future uncertainty for the current constituency
    cluster = con_df["Cluster"].iloc[0]
    imputation_uncertainty = con_df["Low_Confidence_Imputation"].iloc[0]
    
    # Iterate over standard parties
    for p in std_parties:
        # Get the previous results for the current party and constituency
        prev_res = con_df["2019_Results"+p+"_pc"]
        
        # Get the necessary data from the random dataframe
        nat_chg = rand_df[p+"n"]
        clr_chg = rand_df[p+str(cluster)]
        
        # Calculate the noise for the current constituency and party
        con_noise = [np.random.normal(0, (1+imputation_uncertainty)*small_std) for n in range(sims_num)]
        
        # Calculate the rough prediction for the current constituency and party
        prev_res_array = [float(prev_res.iloc[0]) for n in range(sims_num)]
        rough_pred = prev_res_array + nat_chg + clr_chg + con_noise
        rough_pred[rough_pred < 0] = 0
        
        # Store the results in the dictionary
        results[con+p+"Noise"] = con_noise
        results[con+p+"Rough"] = rough_pred
    
# Update the random dataframe with the results
rand_df = pd.concat([rand_df,pd.DataFrame(results)],axis=1)

In [20]:
def get_winner(party_values, winner_val,parties = std_parties):
    for p,v in zip(parties,list(party_values)[0]):
        if v == winner_val:
            return p
def get_margin(party_values):
    sorted_vals = sorted(list(party_values)[0])
    margin = sorted_vals[-1] - sorted_vals[-2]
    return margin

final_dfs = [rand_df]
for con in constituencies:
    con_cols = []
    for p in std_parties:
        con_cols.append(con+p+"Rough")
    final_df = rand_df[con_cols].copy()
    final_pred_cols = []
    for p in std_parties:
        final_df[con+p+"Final"] = final_df[con+p+"Rough"] / final_df[con_cols].sum(axis=1)
        final_pred_cols.append(con+p+"Final")
    final_df[con +"past_post_val"] = final_df[final_pred_cols].max(axis=1)
    final_df[con+"Margin"] = final_df[final_pred_cols].apply(lambda x:get_margin([x[final_pred_cols]]),axis=1)
    final_pred_cols.append(con +"past_post_val")
    final_df[con+"Winner"] = final_df[final_pred_cols].apply(lambda x:get_winner([x[final_pred_cols]],x[con+"past_post_val"]),axis=1)

                
    # final_df[con+"Winner"] = winners
    final_dfs.append(final_df)
rand_df = pd.concat(final_dfs, axis=1)
    

In [21]:
seats_cols = []
for p in std_parties:
    rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
    seats_cols.append(p+"Seats")

/tmp/ipykernel_35709/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_35709/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_35709/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining 

In [22]:
rand_df[seats_cols]

,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats
0,278,291,14,0,48,1
1,304,280,15,1,32,0
2,198,361,26,0,47,0
3,152,410,24,0,46,0
4,202,366,21,0,42,1
...,...,...,...,...,...,...
995,239,300,30,3,58,2
996,287,283,8,0,53,1
997,155,406,18,0,52,1
998,148,357,75,0,50,2


In [23]:
max_seats = rand_df[seats_cols].max(1)
results = []
for n in range(sims_num):
    for col in seats_cols:
        if max_seats[n] ==rand_df[col].loc[n]:
            for p in std_parties:
                if p + "Seats" == col:
                    winning_party = p
    
    if max_seats[n] <326:
        result = "Hung Parliament: Largest Party " + winning_party
    else:
        result = winning_party
    results.append(result)
rand_df["Result"] = results


/tmp/ipykernel_35709/3465883602.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df["Result"] = results


In [24]:
rand_df["Result"].value_counts()

Result
lab                                   780
Hung Parliament: Largest Party lab    185
Hung Parliament: Largest Party con     31
con                                     4
Name: count, dtype: int64

In [25]:
med_df = rand_df[(rand_df["conSeats"]>rand_df["conSeats"].quantile(0.49))&(rand_df["conSeats"]>rand_df["conSeats"].quantile(0.49))]

In [26]:
typical_run = med_df[(med_df["labSeats"]>med_df["labSeats"].quantile(0.4))&(med_df["labSeats"]<med_df["labSeats"].quantile(0.6))].copy()

In [27]:
typical_run

,conn,labn,libn,refn,natn,othn,con0,lab0,lib0,ref0,...,West Dunbartonshirepast_post_val,West DunbartonshireMargin,West DunbartonshireWinner,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats,Result
48,-0.302333,-0.015196,-0.113947,0.110174,-0.029527,0.049667,-0.003513,0.015698,0.042096,-0.012220,...,0.479241,0.174961,nat,211,333,24,2,57,5,lab
53,-0.167052,0.097539,-0.011768,0.072968,-0.073130,-0.011299,-0.037595,-0.000482,-0.028283,0.006668,...,0.485543,0.051558,nat,227,337,25,0,42,1,lab
82,-0.211727,0.050028,0.029780,0.080919,0.053365,0.025726,0.012971,-0.012696,-0.022212,0.003361,...,0.467576,0.150662,nat,207,326,39,1,58,1,lab
85,-0.219379,0.035846,-0.099555,0.069350,0.058396,-0.038361,0.000790,-0.008335,0.008709,0.038240,...,0.551242,0.209351,nat,225,328,18,0,60,1,lab
89,-0.171801,0.119468,-0.080831,0.099604,0.035464,-0.001079,-0.008827,-0.030083,0.019910,-0.002512,...,0.495294,0.090133,nat,231,333,14,0,53,1,lab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,-0.260363,0.020585,-0.016947,0.110685,-0.059256,-0.001224,0.040619,-0.011596,0.019439,0.029169,...,0.495320,0.169384,nat,204,327,47,0,54,0,lab
977,-0.267693,0.023911,0.012240,0.196367,0.007740,0.004609,-0.008004,0.004564,-0.011198,-0.000523,...,0.440308,0.145491,nat,183,329,54,9,56,1,lab
979,-0.187176,0.084742,-0.130613,0.066840,0.076214,-0.013168,0.031844,-0.018765,0.010859,0.003699,...,0.582980,0.238642,nat,232,330,12,0,58,0,lab
987,-0.259237,0.024430,0.023043,0.038929,-0.043012,0.013009,0.000307,-0.017589,-0.016443,0.006075,...,0.536277,0.230568,nat,189,338,51,0,51,3,lab


In [28]:
print(nat_polls["lab"] - nat_polls["con"])

0.21383333333333324


In [29]:
nat_polls

Con         0.234500
Lab         0.448333
Lib Dems    0.089833
SNP         0.030500
Green       0.061167
Reform      0.107667
Others      0.028500
con         0.234500
lab         0.448333
lib         0.089833
ref         0.107667
nat         0.030500
oth         0.028500
dtype: float64

In [30]:
winner_cols = [con+"Winner" for con in constituencies]

In [31]:
winner_wins = []
for col in winner_cols:
    winner_wins.append(max(list(rand_df[col].value_counts())))

In [32]:
close_cols = []
for col,wins in zip(winner_cols,winner_wins):
    if wins < 525:
        close_cols.append(col)

In [33]:
rand_df[close_cols]

,AylesburyWinner,Bicester and WoodstockWinner,"Brecon, Radnor and Cwm TaweWinner",CongletonWinner,Didcot and WantageWinner,DudleyWinner,Gillingham and RainhamWinner,GraveshamWinner,Harpenden and BerkhamstedWinner,HitchinWinner,MansfieldWinner,Ribble ValleyWinner,Rochester and StroodWinner,St Neots and Mid CambridgeshireWinner,StaffordWinner,Sutton and CheamWinner,WokingWinner
0,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con
1,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con
2,con,lib,lab,lab,con,lab,con,con,con,con,lab,lab,con,con,con,lib,con
3,lab,lab,lab,lab,lab,lab,lab,lab,lab,lab,lab,con,lab,lab,con,lib,lab
4,con,con,lab,con,lib,con,con,con,con,con,con,lab,con,con,lab,con,lab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,con,con,con,con,lib,con,con,con,con,con,con,con,con,lib,con,lib,lib
996,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con,con
997,lab,lab,lab,con,lab,lab,lab,lab,lab,lab,lab,con,lab,lab,lab,con,lab
998,con,lib,lib,lab,lib,con,con,con,lib,lib,con,lab,con,lib,lab,lib,lib


In [34]:
all_final_cols = [con+p+"Final"for p in std_parties for con in constituencies]
all_margin_cols = [con+"Margin"for p in std_parties for con in constituencies]
winner_cols
results_cols = ["Result"]

In [35]:
all_result_cols = []
for l in [all_final_cols,all_margin_cols,winner_cols,results_cols]:
    all_result_cols.extend(l)

In [36]:
rand_df[all_result_cols]

,Aberafan MaestegconFinal,AldershotconFinal,Aldridge-BrownhillsconFinal,Altrincham and Sale WestconFinal,Alyn and DeesideconFinal,Amber ValleyconFinal,Arundel and South DownsconFinal,AshfieldconFinal,AshfordconFinal,Ashton-under-LyneconFinal,...,North East FifeWinner,Orkney and ShetlandWinner,Paisley and Renfrewshire NorthWinner,Paisley and Renfrewshire SouthWinner,Perth and Kinross-shireWinner,RutherglenWinner,Stirling and StrathallanWinner,West Aberdeenshire and KincardineWinner,West DunbartonshireWinner,Result
0,0.176431,0.544931,0.620529,0.407466,0.368307,0.630758,0.519798,0.330291,0.634771,0.266118,...,nat,lib,nat,nat,nat,lab,nat,nat,nat,Hung Parliament: Largest Party lab
1,0.198471,0.446994,0.525001,0.356663,0.330313,0.505077,0.469115,0.309599,0.508320,0.252791,...,lib,lib,nat,nat,nat,lab,nat,con,nat,Hung Parliament: Largest Party con
2,0.109769,0.456028,0.514300,0.301093,0.263386,0.456377,0.473028,0.273324,0.494614,0.222463,...,nat,lib,nat,nat,nat,lab,nat,nat,nat,lab
3,0.056068,0.335576,0.467162,0.250971,0.172502,0.436211,0.371005,0.165753,0.389439,0.147080,...,nat,lib,nat,nat,nat,lab,nat,nat,nat,lab
4,0.093245,0.443761,0.497657,0.331956,0.242237,0.477862,0.386844,0.206488,0.482693,0.203710,...,nat,lib,lab,nat,nat,lab,nat,nat,nat,lab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.037822,0.417197,0.494346,0.312893,0.197349,0.480884,0.432547,0.191415,0.458838,0.113859,...,nat,lib,nat,nat,nat,nat,nat,nat,nat,Hung Parliament: Largest Party lab
996,0.151544,0.448109,0.519693,0.382634,0.294300,0.516477,0.458516,0.256238,0.595256,0.249079,...,nat,nat,nat,nat,nat,nat,nat,nat,nat,Hung Parliament: Largest Party con
997,0.008813,0.425730,0.464649,0.269204,0.159517,0.437968,0.342454,0.150879,0.490463,0.128684,...,nat,lib,nat,nat,nat,nat,nat,nat,nat,lab
998,0.001009,0.340349,0.366614,0.208017,0.156714,0.353513,0.312968,0.123894,0.367889,0.075997,...,lib,lib,nat,nat,nat,nat,nat,nat,nat,lab


In [37]:
all_margin_cols

['Aberafan MaestegMargin',
 'AldershotMargin',
 'Aldridge-BrownhillsMargin',
 'Altrincham and Sale WestMargin',
 'Alyn and DeesideMargin',
 'Amber ValleyMargin',
 'Arundel and South DownsMargin',
 'AshfieldMargin',
 'AshfordMargin',
 'Ashton-under-LyneMargin',
 'AylesburyMargin',
 'BanburyMargin',
 'Bangor AberconwyMargin',
 'BarkingMargin',
 'Barnsley NorthMargin',
 'Barnsley SouthMargin',
 'Barrow and FurnessMargin',
 'Basildon and BillericayMargin',
 'BasingstokeMargin',
 'BassetlawMargin',
 'BathMargin',
 'BatterseaMargin',
 'BeaconsfieldMargin',
 'Beckenham and PengeMargin',
 'BedfordMargin',
 'Bermondsey and Old SouthwarkMargin',
 'Bethnal Green and StepneyMargin',
 'Beverley and HoldernessMargin',
 'Bexhill and BattleMargin',
 'Bexleyheath and CrayfordMargin',
 'Bicester and WoodstockMargin',
 'BirkenheadMargin',
 'Birmingham EdgbastonMargin',
 'Birmingham ErdingtonMargin',
 'Birmingham Hall Green and MoseleyMargin',
 'Birmingham Hodge Hill and Solihull NorthMargin',
 'Birmingha